In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [2]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def drawLine(img, x, y, color=[255, 0, 0], thickness=20):
    """
    Adjust a line to the points [`x`, `y`] and draws it on the image `img` using `color` and `thickness` for the line.
    """
    

def draw_lines(img, lines, color=[255, 0, 0], thickness=20):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    left_X=[]
    left_Y=[]
    right_X=[]
    right_Y=[]
    ##To separate the points between left and right side
    for line in lines:
        for x1,y1,x2,y2 in line:
            m = (y1 - y2)/(x1 - x2)
            if m<0:
                left_X.append(x1)
                left_Y.append(y1)
                left_X.append(x2)
                left_Y.append(y2)
            else:
                right_X.append(x1)
                right_Y.append(y1)
                right_X.append(x2)
                right_Y.append(y2)

    if len(left_X) != 0: 
        #Drawing the lines with the given slope
        p = np.polyfit(left_X, left_Y, 1) 
        y_start = img.shape[0] #Line must start from the bottom of the image
        x_start = int((y_start - p[1])/p[0])
        y_end = 350 #Goes upto this y axis on the image
        x_end = int((y_end - p[1])/p[0])
        cv2.line(img, (x_start, y_start), (x_end, y_end), [255, 0, 0], 10)
    
    if len(right_X) != 0: 
    #Drawing the lines with the given slope
        p = np.polyfit(right_X, right_Y, 1)  
        y_start = img.shape[0] #Line must start from the bottom of the image
        x_start = int((y_start - p[1])/p[0])
        y_end = 350 #Goes upto this y axis on the image
        x_end = int((y_end - p[1])/p[0])
        cv2.line(img, (x_start, y_start), (x_end, y_end), [0, 255, 0], 10)
        

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

In [3]:
def lane_pipeline(image):
    init_image = image.copy()
    image=grayscale(image)
    image=gaussian_blur(image,5)
    image=canny(image,50,150)
    imshape = image.shape
    vertices=np.array([[(0,imshape[0]),(450, 350), (imshape[1]-470, 330), (imshape[1],imshape[0])]], dtype=np.int32)
    image=region_of_interest(image,vertices)
    image=hough_lines(image, 2, 3*np.pi/180, 40, 60, 200)
    image=weighted_img(image,init_image)
    return image




In [4]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
white_output = 'solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(lane_pipeline) #NOTE: this function expects color images!!

In [5]:
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video solidWhiteRight.mp4
[MoviePy] Writing video solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:03<00:00, 64.18it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: solidWhiteRight.mp4 

CPU times: user 1.63 s, sys: 456 ms, total: 2.09 s
Wall time: 4.01 s
